In [252]:
import numpy as np
import pandas as pd
from pathlib import Path
import datetime
import warnings
import numpy as np
import json
import copy
import urllib.request
warnings.filterwarnings("ignore")
pd.options.display.max_columns = 500
pd.options.display.max_rows = 1000

In [289]:
# import data
dir = str(Path().resolve())
air = pd.read_csv(dir + "/../processing/data/airbnb/counts.csv")
apts = pd.read_csv(dir + "/../processing/data/apts/counts.csv")
with urllib.request.urlopen("http://bostonopendata-boston.opendata.arcgis.com/datasets/53ea466a189b4f43b3dfb7b38fa7f3b6_1.geojson") as url:
    zips = json.loads(url.read().decode())

In [290]:
air = air.drop('counts', axis=1)
apts = apts.drop('counts', axis=1)

In [291]:
air['density'] = air['density'] / air.density.max()
apts['density'] = apts['density'] / apts.density.max()

In [292]:
apts['zipcode'] = "0" + apts['zipcode'].apply(str)

In [293]:
apt_geo = copy.deepcopy(zips)
air_geo = copy.deepcopy(zips)

In [294]:
for y in apt_geo['features']:
    zc = y['properties']['ZIP5']
    if zc in apts.zipcode.unique():
        y['properties']['density'] = apts.density.iloc[apts.zipcode[apts.zipcode==zc].index[0]]
    else:
        y['properties']['density'] = 0

In [295]:
for z in air_geo['features']:
    zc = z['properties']['ZIP5']
    if zc in air.zipcode.unique():
        z['properties']['density'] = air.density.iloc[air.zipcode[air.zipcode==zc].index[0]]
    else:
        z['properties']['density'] = 0

In [296]:
with open('apts.json', 'w') as outfile:  
    json.dump(apt_geo, outfile)

In [298]:
with open('airs.json', 'w') as outfile:  
    json.dump(air_geo, outfile)